Ошибки посмотреть под ячейками  
На веб морде эйрфлоу предупреждение: "The scheduler does not appear to be running. The DAGs list may not update, and new tasks will not be scheduled".

Код из статей:
- [Как запустить Apache AirFlow в Google Colab с ngrok](https://bigdataschool.ru/blog/how-to-launch-airflow-on-colab-with-ngrok.html)
- [Как запустить DAG AirFlow в Google Colab: простой пример](https://bigdataschool.ru/blog/how-to-run-user-dag-airflow-in-colab-with-ngrok.html) (перепроверить работу начала второй части и продолжить с Запуск пользовательского DAG)

In [2]:
#############################ячейка №1 в Google Colab###########################
#Установка Apache Airflow и инициализация базы данных
!pip install apache-airflow
!airflow db init
## initdb - было так в оригинале, но пишет об ошибке и замене команды
#Установка инструмента ngrok для создания безопасного туннеля
#для доступа к веб-интерфейсу Airflow из любого места
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!pip install pyngrok
#импорт модулей
from pyngrok import ngrok
import sys
import os
import datetime
from airflow import DAG
from airflow.operators.bash import BashOperator

from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/airflow/dags', exist_ok=True)
dags_folder = os.path.join(os.path.expanduser("~"), "airflow", "dags")
os.makedirs(dags_folder, exist_ok=True)

DB: sqlite:////root/airflow/airflow.db
[2023-11-25T07:43:38.553+0000] {migration.py:213} INFO - Context impl SQLiteImpl.
[2023-11-25T07:43:38.554+0000] {migration.py:216} INFO - Will assume non-transactional DDL.
[2023-11-25T07:43:39.084+0000] {migration.py:213} INFO - Context impl SQLiteImpl.
[2023-11-25T07:43:39.085+0000] {migration.py:216} INFO - Will assume non-transactional DDL.
[2023-11-25T07:43:39.086+0000] {db.py:1620} INFO - Creating tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
WARNI [unusual_prefix_13903d1433e73a905598409829be4c61c9cdea5f_example_kubernetes_executor] The example_kubernetes_executor example DAG requires the kubernetes provider. Please install it with: pip install apache-airflow[cncf.kubernetes]
WARNI [unusual_prefix_4c1c0063a36d033a56d4343efc723c3a45c1bd59_example_local_kubernetes_executor] Could not import DAGs in example_local_kubernetes_executor.py
Traceback (most rece

In [3]:
#######################ячейка №2 в Google Colab########################
#Задание переменной auth_token для аутентификации в сервисе ngrok.
auth_token = "2WhKx82LS1maMJ2dMagXgh9ipsb_3fcJGdvLkWMHihaHRpLju" #@param {type:"string"}
# Since we can't access Colab notebooks IP directly we'll use
# ngrok to create a public URL for the server via a tunnel
# Authenticate ngrok
# https://dashboard.ngrok.com/signup
# Then go to the "Your Authtoken" tab in the sidebar
# and copy the API key
#Аутентификация в сервисе ngrok с помощью auth_token
os.system(f"ngrok authtoken {auth_token}")
#Запуск ngrok, который создаст публичный URL для сервера через туннель
#для доступа к веб-интерфейсу Airflow из любого места.
#addr="8888" указывает на порт, на котором запущен веб-сервер Airflow,
#а proto="http" указывает на использование протокола HTTP
public_url = ngrok.connect(addr="8888", proto="http")
#Вывод публичного URL для доступа к веб-интерфейсу Airflow
print("Адрес Airflow GUI:", public_url)

[2023-11-25T07:47:26.883+0000] {ngrok.py:276} INFO - Opening tunnel named: http-8888-f417358d-ad5f-4354-9d35-77d34781873e
[2023-11-25T07:47:26.948+0000] {process.py:99} INFO - t=2023-11-25T07:47:26+0000 lvl=info msg="no configuration paths supplied"
[2023-11-25T07:47:26.955+0000] {process.py:99} WARNING - t=2023-11-25T07:47:26+0000 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=/root/.config/ngrok/ngrok.yml legacy_path=/root/.ngrok2/ngrok.yml
[2023-11-25T07:47:26.958+0000] {process.py:99} INFO - t=2023-11-25T07:47:26+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
[2023-11-25T07:47:26.967+0000] {process.py:99} INFO - t=2023-11-25T07:47:26+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
[2023-11-25T07:47:26.971+0000] {process.py:99} INFO - t=2023-11-25T07:47:26+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
[2023-11-25T07:47:27.033+0000] {p

In [4]:
#############################ячейка №3 в Google Colab#########################
!airflow db init #Инициализация базы данных Airflow  ## это же было в первой ячейке
!airflow db upgrade #Обновление базы данных Airflow
## !airflow upgradedb - в оригинале было, но ворнинги требуют новый синтаксис
#Создание нового пользователя в Apache Airflow с именем пользователя anna, именем Anna, фамилией Anna, адресом электронной почты anna.doe@example.com и паролем password.
#Этот пользователь будет иметь роль Admin, которая дает полный доступ к интерфейсу Airflow.
!airflow users create --username anna --firstname Anna --lastname Anna --email anna.doe@example.com --role Admin --password password

DB: sqlite:////root/airflow/airflow.db
[2023-11-25T07:50:05.190+0000] {migration.py:213} INFO - Context impl SQLiteImpl.
[2023-11-25T07:50:05.191+0000] {migration.py:216} INFO - Will assume non-transactional DDL.
[2023-11-25T07:50:05.722+0000] {migration.py:213} INFO - Context impl SQLiteImpl.
[2023-11-25T07:50:05.722+0000] {migration.py:216} INFO - Will assume non-transactional DDL.
[2023-11-25T07:50:05.724+0000] {db.py:1620} INFO - Creating tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
WARNI [unusual_prefix_13903d1433e73a905598409829be4c61c9cdea5f_example_kubernetes_executor] The example_kubernetes_executor example DAG requires the kubernetes provider. Please install it with: pip install apache-airflow[cncf.kubernetes]
WARNI [unusual_prefix_4c1c0063a36d033a56d4343efc723c3a45c1bd59_example_local_kubernetes_executor] Could not import DAGs in example_local_kubernetes_executor.py
Traceback (most rece

In [ ]:
## выше адрес вида https://2890-34-69-125-71.ngrok-free.app,
## по которому  зайдем в веб-интерфейс Apache AirFlow

In [5]:
##########################ячейка №5 в Google Colab#######################
#запуск веб-сервера Apache Airflow на порту 8888. Веб-сервер Airflow предоставляет пользовательский интерфейс
#для управления DAG’ами,
#просмотра логов выполнения задач, мониторинга прогресса выполнения
!airflow webserver --port 8888

[2023-11-25T07:50:42.416+0000] {configuration.py:2049} INFO - Creating new FAB webserver config file in: /root/airflow/webserver_config.py
  ____________       _____________
 ____    |__( )_________  __/__  /________      __
____  /| |_  /__  ___/_  /_ __  /_  __ \_ | /| / /
___  ___ |  / _  /   _  __/ _  / / /_/ /_ |/ |/ /
 _/_/  |_/_/  /_/    /_/    /_/  \____/____/|__/
Running the Gunicorn Server with:
Workers: 4 sync
Host: 0.0.0.0:8888
Timeout: 120
Logfiles: - -
Access Logformat: 
/usr/local/lib/python3.10/dist-packages/flask_limiter/extension.py:336 UserWarning: Using the in-memory storage for tracking rate limits as no storage was explicitly specified. This is not recommended for production use. See: https://flask-limiter.readthedocs.io#configuring-a-storage-backend for documentation about configuring the storage backend.
[2023-11-25T07:50:45.342+0000] {manager.py:499} INFO - Created Permission View: %s
[2023-11-25T07:50:45.355+0000] {manager.py:555} INFO - Added Permission %s to

In [ ]:
## Теперь можно войти в веб-интерфейс Apache AirFlow,
## по ссылке, ранее сгенерированной утилитой ngrok.
## Надо ввести логин и пароль пользователя, созданного ранее: anna, password

-------------

In [3]:
###############################################ячейка №1 в Google Colab#############################
#Установка Apache Airflow и инициализация базы данных.
!pip install apache-airflow
## !airflow init  ## еще будет в следующей ячейке
## initdb - было так в оригинале, но пишет об ошибке и замене команды
#Установка инструмента ngrok для создания безопасного туннеля для доступа к веб-интерфейсу Airflow из любого места
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!pip install pyngrok

!pip install sqlalchemy  ## ругается установка на конфликт со старой версией алхимии

#импорт модулей
from pyngrok import ngrok
import sys
import os
import datetime
from airflow import DAG
from airflow.operators.bash import BashOperator

## свой dag будем хранить на гугл-драйв
from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/airflow/dags', exist_ok=True)
dags_folder = os.path.join(os.path.expanduser("~"), "airflow", "dags")
os.makedirs(dags_folder, exist_ok=True)

--2023-11-25 08:40:21--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 52.202.168.65, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.28M  87.0MB/s    in 0.2s    

2023-11-25 08:40:22 (87.0 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
###############################################ячейка №2 в Google Colab#############################
#Задание переменной auth_token для аутентификации в сервисе ngrok.
auth_token = "2WhKx82LS1maMJ2dMagXgh9ipsb_3fcJGdvLkWMHihaHRpLju"
# Since we can't access Colab notebooks IP directly we'll use
# ngrok to create a public URL for the server via a tunnel
# Authenticate ngrok
# https://dashboard.ngrok.com/signup
# Then go to the "Your Authtoken" tab in the sidebar and copy the API key
#Аутентификация в сервисе ngrok с помощью auth_token
os.system(f"ngrok authtoken {auth_token}")
#Запуск ngrok, который создаст публичный URL для сервера через туннель
#для доступа к веб-интерфейсу Airflow из любого места.
#addr="8888" указывает на порт, на котором запущен веб-сервер Airflow, а proto="http" указывает на использование протокола HTTP
public_url = ngrok.connect(addr="8888", proto="http")
#Вывод публичного URL для доступа к веб-интерфейсу Airflow
print("Адрес Airflow GUI:", public_url)

[2023-11-25T08:40:55.045+0000] {ngrok.py:276} INFO - Opening tunnel named: http-8888-131c7a9b-f741-4691-97a3-11c45e251c30
[2023-11-25T08:40:55.080+0000] {process.py:99} INFO - t=2023-11-25T08:40:55+0000 lvl=info msg="no configuration paths supplied"
[2023-11-25T08:40:55.083+0000] {process.py:99} WARNING - t=2023-11-25T08:40:55+0000 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=/root/.config/ngrok/ngrok.yml legacy_path=/root/.ngrok2/ngrok.yml
[2023-11-25T08:40:55.085+0000] {process.py:99} INFO - t=2023-11-25T08:40:55+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
[2023-11-25T08:40:55.087+0000] {process.py:99} INFO - t=2023-11-25T08:40:55+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
[2023-11-25T08:40:55.089+0000] {process.py:99} INFO - t=2023-11-25T08:40:55+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
[2023-11-25T08:40:55.168+0000] {p

In [5]:
## ip для входа чуть выше в формате типа https://ec27-34-86-112-48.ngrok-free.app

In [6]:
#############################ячейка №3 в Google Colab#########################
!airflow db init #Инициализация базы данных Airflow  ## это же было в первой ячейке
!airflow db upgrade #Обновление базы данных Airflow
## !airflow upgradedb - в оригинале было, но ворнинги требуют новый синтаксис
#Создание нового пользователя в Apache Airflow с именем пользователя anna, именем Anna, фамилией Anna, адресом электронной почты anna.doe@example.com и паролем password.
#Этот пользователь будет иметь роль Admin, которая дает полный доступ к интерфейсу Airflow.
!airflow users create --username anna --firstname Anna --lastname Anna --email anna@example.com --role Admin --password password

DB: sqlite:////root/airflow/airflow.db
[2023-11-25T08:43:10.294+0000] {migration.py:213} INFO - Context impl SQLiteImpl.
[2023-11-25T08:43:10.295+0000] {migration.py:216} INFO - Will assume non-transactional DDL.
[2023-11-25T08:43:10.831+0000] {migration.py:213} INFO - Context impl SQLiteImpl.
[2023-11-25T08:43:10.832+0000] {migration.py:216} INFO - Will assume non-transactional DDL.
[2023-11-25T08:43:10.833+0000] {db.py:1620} INFO - Creating tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
WARNI [unusual_prefix_13903d1433e73a905598409829be4c61c9cdea5f_example_kubernetes_executor] The example_kubernetes_executor example DAG requires the kubernetes provider. Please install it with: pip install apache-airflow[cncf.kubernetes]
WARNI [unusual_prefix_4c1c0063a36d033a56d4343efc723c3a45c1bd59_example_local_kubernetes_executor] Could not import DAGs in example_local_kubernetes_executor.py
Traceback (most rece

In [7]:
import random
import math
import os
from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from airflow.operators.python_operator import PythonOperator
from google.colab import files
from datetime import datetime, timedelta
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': datetime.now(),
    'email_on_failure': True,
    'email_on_success': True,
    'email': ['vichugova.anna@gmail.com'],
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}
dag = DAG(
    'ANNA_DAG_SaveFile',
    default_args=default_args,
    schedule_interval=timedelta(days=1),
)
def write_to_file(**context):
    execution_date = context['execution_date'].strftime('%Y-%m-%d')
    result = context['ti'].xcom_pull(task_ids='task4')
    filename = f'/content/airflow/results/{execution_date}.txt'
    with open(filename, 'a') as f:
        f.write(result + '\n')
# Ниже определяются операторы, которые будут выполняться в DAG
# Формирование текста с номером запуска DAG в task1, вывод текущей даты в task2,
# вывод текущего времени в task3, вывод результатов выполнения task2 и task3 в task4,
# и запись результатов в txt-файл с названием текущей даты в task5
task1 = BashOperator(
    task_id='task1',
    bash_command='echo "Запуск DAG {{ dag_run.run_id }}" | tee /tmp/dag_run_id.txt',
    dag=dag,
)
task2 = BashOperator(
    task_id='task2',
    bash_command='echo "Сегодня $(date +%d-%m-%Y)"',
    dag=dag,
)
task3 = BashOperator(
    task_id='task3',
    bash_command='echo "В $(date +%H:%M:%S)"',
    dag=dag,
)
task4 = BashOperator(
    task_id='task4',
    bash_command='echo "{{ ti.xcom_pull(task_ids=\'task1\') }}. {{ ti.xcom_pull(task_ids=\'task2\') }}. {{ ti.xcom_pull(task_ids=\'task3\') }}"',
    dag=dag,
)
task5 = PythonOperator(
    task_id='task5',
    python_callable=write_to_file,
    dag=dag
)
task1 >> [task2, task3] >> task4 >> task5

<ipython-input-7-28c7cfc05103>:5 DeprecationWarning: The `airflow.operators.bash_operator.BashOperator` class is deprecated. Please use `'airflow.operators.bash.BashOperator'`.

<ipython-input-7-28c7cfc05103>:6 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

<ipython-input-7-28c7cfc05103>:19 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

<Task(PythonOperator): task5>

In [ ]:
!cp /content/airflow/dags/ANNA_DAG_SaveFile.py ~/airflow/dags/ANNA_DAG_SaveFile.py
!airflow dags unpause ANNA_DAG_SaveFile

In [ ]:
!airflow webserver --port 8888 --daemon

In [ ]:
!airflow scheduler --daemon

In [ ]:
!pkill -f ngrok #поиск и завершение процесса, который запущен с помощью команды ngrok
# отключение публичного URL, созданного с помощью ngrok для веб-сервера Airflow
ngrok.disconnect(public_url=public_url)

In [ ]:
############################ячейка №6 в Google Colab############################
!pkill -f ngrok #поиск и завершение процесса, который запущен с помощью команды ngrok
# отключение публичного URL, созданного с помощью ngrok для веб-сервера Airflow
ngrok.disconnect(public_url=public_url)